In [1]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

learning_rate = 0.1
num_epochs = 10
batch_size = 128

In [25]:
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [27]:
for image , label in train_loader:
    print('Image batch dimensions:' , image.shape)
    print('Image label dimensions:' , label.shape)
    break
print(len(test_loader))

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])
79


In [28]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet,self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool_1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2), padding=0)
        self.conv_2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=1)
        self.pool_2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2), padding=0)
        self.linear_1 = nn.Linear(7*7*16, num_classes)
        
    def forward(self,x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)
        
        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        out = self.linear_1(out.view(-1,7*7*16))
        pro = F.softmax(out, dim=1)
        return pro

torch.manual_seed(1)
model = ConvNet(10)

model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
      

In [29]:
def metric_acc(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0,0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            pro = model(features)
            _, pre = torch.max(pro, 1)
            num_examples += targets.size(0)
            correct_pred += (pre==targets).sum()
        return correct_pred.float()/num_examples *100

In [31]:
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_ind, (features, targets) in enumerate(train_loader):
        features = features.to(device)
        targets = targets.to(device)
        
        pro = model(features)
        loss = F.cross_entropy(pro, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if not batch_ind%50:
            print('Epoch %03d/%03d | Batch %03d/%03d | Loss: %.4f' % ((epoch+1), num_epochs , batch_ind, 
                  len(train_loader), loss))
    with torch.no_grad():
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              metric_acc(model, train_loader)))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))   

Epoch 001/010 | Batch 000/469 | Loss: 2.3040
Epoch 001/010 | Batch 050/469 | Loss: 2.3002
Epoch 001/010 | Batch 100/469 | Loss: 2.2966
Epoch 001/010 | Batch 150/469 | Loss: 2.2811
Epoch 001/010 | Batch 200/469 | Loss: 2.1769
Epoch 001/010 | Batch 250/469 | Loss: 1.8251
Epoch 001/010 | Batch 300/469 | Loss: 1.8491
Epoch 001/010 | Batch 350/469 | Loss: 1.7781
Epoch 001/010 | Batch 400/469 | Loss: 1.7344
Epoch 001/010 | Batch 450/469 | Loss: 1.8122
Epoch: 001/010 training accuracy: 71.04%
Time elapsed: 0.13 min
Epoch 002/010 | Batch 000/469 | Loss: 1.8571
Epoch 002/010 | Batch 050/469 | Loss: 1.7402
Epoch 002/010 | Batch 100/469 | Loss: 1.7255
Epoch 002/010 | Batch 150/469 | Loss: 1.7450
Epoch 002/010 | Batch 200/469 | Loss: 1.6342
Epoch 002/010 | Batch 250/469 | Loss: 1.7062
Epoch 002/010 | Batch 300/469 | Loss: 1.6978
Epoch 002/010 | Batch 350/469 | Loss: 1.6770
Epoch 002/010 | Batch 400/469 | Loss: 1.6821
Epoch 002/010 | Batch 450/469 | Loss: 1.7381
Epoch: 002/010 training accuracy: 75

In [32]:
print('Test accuracy: %.2f%%' % (metric_acc(model, test_loader)))

Test accuracy: 78.97%
